## Top

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
!nvidia-smi

Tue Aug 15 13:27:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [ ]:
train_path = "/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/DATA/Full_train_test/pre_processed_binarized/train_full_binarized_pre.csv"
test_path = "/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/DATA/Full_train_test/pre_processed_binarized/test_off_binarized_pre.csv"

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
train_df.tail()

,tweet,side-effect,ineffective,rushed,pharma,mandatory,unnecessary,political,ingredients,conspiracy,country,religious
9916,former pfizer chief scientific officer experi...,1,0,0,0,0,0,0,0,0,0,0
9917,manufacturers saying manufacturers dont recom...,0,0,0,1,0,0,0,0,0,0,0
9918,thats complete oxfordastrazeneca vaccine swis...,0,0,0,0,0,0,0,0,0,0,0
9919,opinion vaccine side effects possible penicil...,1,0,0,0,0,0,0,0,0,0,0
9920,resign destroy country live 50 adults double ...,0,0,0,0,0,0,1,0,0,0,0


### Run from here

In [ ]:
len(train_df)

9921

In [ ]:
target_list = ['side-effect', 'ineffective', 'rushed', 'pharma', 'mandatory',
       'unnecessary', 'political', 'ingredients', 'conspiracy', 'country',
       'religious']

In [ ]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 4
LEARNING_RATE = 1e-05

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('digitalepidemiologylab/covid-twitter-bert-v2')

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['tweet']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

Dont run this cell repeatedly

In [ ]:
train_size = 0.8
df = train_df
train_df = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = df.drop(train_df.index).reset_index(drop=True)

In [ ]:
len(train_df)

7937

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)
train_dataset.__getitem__(0)
#len(valid_dataset)

{'input_ids': tensor([  101, 22552,  2529, 14471, 12436, 14693,  9323,  2336,  2015,  2740,
          3639,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
len(train_data_loader)

794

In [ ]:
def return_label_wts(df):
  all_labels_count = df.iloc[:, 1:].sum()
  total_samples = len(df)
  weights = torch.tensor(total_samples) / (torch.tensor(all_labels_count) + 1e-6)

  return weights

In [ ]:
train_weights = torch.tensor(return_label_wts(train_df))
test_weights = torch.tensor(return_label_wts(test_df))
val_weights = torch.tensor(return_label_wts(val_df))

<ipython-input-50-f44562345109>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_weights = torch.tensor(return_label_wts(train_df))
<ipython-input-50-f44562345109>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_weights = torch.tensor(return_label_wts(test_df))
<ipython-input-50-f44562345109>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_weights = torch.tensor(return_label_wts(val_df))


In [ ]:
train_weights

tensor([  2.6186,   5.9857,   6.7664,   7.7058,  12.6789,  13.6375,  15.8108,
         21.8650,  20.0937,  48.6933, 168.8723])

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('digitalepidemiologylab/covid-twitter-bert-v2', return_dict=True)
        self.dropout = torch.nn.Dropout(0.5)
        self.linear = torch.nn.Linear(1024, 11)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, 

In [ ]:
def loss_fn(outputs, targets, weights):
    return torch.nn.BCEWithLogitsLoss(weight=weights)(outputs, targets)

optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)

In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  global train_weights, val_weights
  train_weights, val_weights = train_weights.to(device, dtype=torch.float), val_weights.to(device, dtype=torch.float)
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0
    t0 = time.time()
    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))

    for batch_idx, data in enumerate(training_loader):
    # Progress update every 40 batches.
      if batch_idx % 40 == 0 and not batch_idx == 0:
          # Calculate elapsed time in minutes.
          elapsed = format_time(time.time() - t0)
          # Report progress.
          print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(batch_idx, len(training_loader), elapsed))
      #print('yyy epoch', batch_idx)

      ids = data['input_ids'].to(device, dtype = torch.long)
      mask = data['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      targets = data['targets'].to(device, dtype = torch.float)

      outputs = model(ids, mask, token_type_ids)

      optimizer.zero_grad()
      loss = loss_fn(outputs, targets, weights = train_weights)
      #if batch_idx%5000==0:
        #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      #print('before loss data in training', loss.item(), train_loss)
      train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
      #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets, weights = val_weights)
        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
        val_targets.extend(targets.cpu().detach().numpy().tolist())
        val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
ckpt_path = "/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/BERT/ckpts/curr_ckpt"
best_model_path = "/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/covid-twitter-bert/model_wt_0.5dpout.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
  Batch    40  of    794.    Elapsed: 0:00:54.
  Batch    80  of    794.    Elapsed: 0:01:49.
  Batch   120  of    794.    Elapsed: 0:02:44.
  Batch   160  of    794.    Elapsed: 0:03:38.
  Batch   200  of    794.    Elapsed: 0:04:33.
  Batch   240  of    794.    Elapsed: 0:05:27.
  Batch   280  of    794.    Elapsed: 0:06:22.
  Batch   320  of    794.    Elapsed: 0:07:17.
  Batch   360  of    794.    Elapsed: 0:08:11.
  Batch   400  of    794.    Elapsed: 0:09:06.
  Batch   440  of    794.    Elapsed: 0:10:01.
  Batch   480  of    794.    Elapsed: 0:10:55.
  Batch   520  of    794.    Elapsed: 0:11:50.
  Batch   560  of    794.    Elapsed: 0:12:44.
  Batch   600  of    794.    Elapsed: 0:13:39.
  Batch   640  of    794.    Elapsed: 0:14:33.
  Batch   680  of    794.    Elapsed: 0:15:28.
  Batch   720  of    794.    Elapsed: 0:16:23.
  Batch   760  of    794.    Elapsed: 0:17:17.
############# Epoch 1: Training End     #############

In [ ]:
# testing
example = test_df['tweet'][3]
print(example)
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
    print(str(final_output) + '\n', train_df.columns[1:].to_list())

 honestly made invisible enemy herd immunity worldwide without vaccine much afraid vaccine imo
mandatory
[[0.041843440383672714, 0.5294681191444397, 0.02123350463807583, 0.02301955223083496, 0.10925135016441345, 0.6523628234863281, 0.01970646157860756, 0.030256571248173714, 0.04786137118935585, 0.02149927243590355, 0.008819708600640297, 0.03396201133728027]]
 ['tweet', 'side-effect', 'ineffective', 'rushed', 'pharma', 'mandatory', 'unnecessary', 'political', 'ingredients', 'conspiracy', 'country', 'religious', 'none']


# Evaluation

## Load model

In [5]:
import torch
from transformers import BertTokenizer, BertModel


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [6]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min

In [7]:
# Specify the path to the .pt file
model_path = "/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/covid-twitter-bert/models/model_initial.pt"

LEARNING_RATE = 1e-05
MAX_LEN = 256

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('digitalepidemiologylab/covid-twitter-bert-v2', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(1024, 11)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
tokenizer = BertTokenizer.from_pretrained('digitalepidemiologylab/covid-twitter-bert-v2')
model, optimizer, epoch, valid_loss_min = load_ckp(model_path, model, optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE))

model.to(device)
model.eval()

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, 

In [8]:
valid_loss_min

0.00040782630539619105

In [9]:
import numpy as np
from sklearn.metrics import f1_score
import pandas as pd

def return_vec(text):
  encodings = tokenizer.encode_plus(
      text,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model(input_ids, attention_mask, token_type_ids)
      final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
  return final_output[0]


def find_best_thresholds(input_vectors, desired_outputs):
    num_labels = len(desired_outputs[0])
    threshold_list = []

    for label in range(num_labels):
        best_threshold = None
        best_f1_score = 0.0

        for threshold in np.linspace(0, 1, num=100):
            converted_outputs = [1 if val[label] >= threshold else 0 for val in input_vectors]
            f1 = f1_score(np.array(desired_outputs)[:, label], converted_outputs, average='macro')

            if f1 > best_f1_score:
                best_f1_score = f1
                best_threshold = threshold

        threshold_list.append(round(best_threshold, 3))

    return threshold_list


def filter_threshold_lst(input_vectors, threshold_list):
    optimized_vectors = []

    for vector in input_vectors:
        optimized_vector = np.array([1 if val >= threshold else 0 for val, threshold in zip(vector, threshold_list)])
        optimized_vectors.append(optimized_vector)

    return optimized_vectors


def generate_y_true(test_csv_path):
  # Read the CSV file into a DataFrame
  df = pd.read_csv(test_csv_path)

  # Extract values excluding the first column
  result = df.iloc[:, 1:].values.tolist()
  result_np = [np.array(sublist) for sublist in result]
  return result_np

def generate_y_pred(test_csv_path):
  df = pd.read_csv(test_csv_path)
  y_pred = []
  for i in range(len(df)):
    y_pred.append(np.array(return_vec(df['tweet'][i])))
  return y_pred


def predict(text, threshold_lst):
  pred_lbl_lst = []
  labels = ('side-effect', 'ineffective', 'rushed', 'pharma', 'mandatory', 'unnecessary', 'political', 'ingredients', 'conspiracy', 'country', 'religious', 'none')
  prob_lst = return_vec(text)
  vec = filter_threshold_lst_1D(prob_lst, threshold_lst)
  #vec = filter_threshold_1D(prob_lst, threshold)
  if prob_lst[11] == max(prob_lst) or vec[:11] == [0] * 11:
    pred_lbl_lst = ['none']
    vec = [0] * 11
    vec.append(1)
    return pred_lbl_lst, vec, prob_lst

  if vec[-1] == 1 and sum(vec[:-1]) > 0:
    vec[-1] = 0
  for i in range(len(vec)):
    if vec[i]==1:
      pred_lbl_lst.append(labels[i])
  return pred_lbl_lst, vec, prob_lst


def predict_csv(test_csv_path, threshold_lst):
  df = pd.read_csv(test_csv_path)
  y_pred = []
  for i in range(len(df)):
    pred_lbl_lst, vec, prob_lst = predict(df['tweet'][i], threshold_lst)
    y_pred.append(np.array(vec))

  return y_pred



In [ ]:
thres = [0.586,0.434,0.576,0.081,0.596,0.414,0.242,0.202,0.323,0.091,0.061]
#thres = 0.23
predict(' thats like arlene come right say one famous redlines vaccines ireland cos give vaccine keep coming back looking', thres)

NameError: ignored

In [10]:
test_csv_path = '/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/DATA/Full_train_test/pre_processed_binarized/test_off_binarized_pre.csv'
y_pred = generate_y_pred(test_csv_path)
y_true = generate_y_true(test_csv_path)

In [ ]:
y_pred


[array([0.98924035, 0.00821668, 0.01362231, 0.00461543, 0.00502472,
        0.00493413, 0.00480244, 0.00740358, 0.0065043 , 0.00305356,
        0.00318753]),
 array([0.97131866, 0.00674993, 0.01195832, 0.00524621, 0.00703608,
        0.00373839, 0.00578762, 0.14726256, 0.01208386, 0.00264179,
        0.00628202]),
 array([0.98942357, 0.0063076 , 0.00935267, 0.00725594, 0.00509169,
        0.00415591, 0.00720822, 0.00757243, 0.00499931, 0.00241759,
        0.00347326]),
 array([0.99207723, 0.00634688, 0.00876154, 0.00744325, 0.00523378,
        0.0051048 , 0.00668349, 0.00856347, 0.00704525, 0.00245205,
        0.00340859]),
 array([0.84992296, 0.02918655, 0.00490457, 0.00252729, 0.00242367,
        0.00397431, 0.00737394, 0.01864921, 0.01144453, 0.00317059,
        0.00347193]),
 array([0.9852159 , 0.00645342, 0.02435707, 0.0401018 , 0.00277465,
        0.00422279, 0.00974062, 0.01366705, 0.00756656, 0.00182788,
        0.00244255]),
 array([0.4730812 , 0.00615962, 0.01146613, 0.956084

In [ ]:
threshold_lst = find_best_thresholds(y_pred, y_true)
threshold_lst

[0.616, 0.212, 0.051, 0.131, 0.212, 0.111, 0.071, 0.566, 0.061, 0.02, 0.081]

In [12]:
threshold_lst = [0.616, 0.212, 0.051, 0.131, 0.212, 0.111, 0.071, 0.566, 0.061, 0.02, 0.081]

In [13]:
#threshold_lst = [0.3] * 11
y_pred_filt = filter_threshold_lst(y_pred, threshold_lst)
y_pred_filt

[array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]),
 array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]),
 array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 

### Generate prediction csv

In [56]:
test_raw_path = '/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/DATA/test_full_raw/test.csv'
test_raw_df = pd.read_csv(test_raw_path)
test_raw_df = test_raw_df.drop(['tweet'], axis=1)
test_raw_df.head()

,id
0,1070378532260470789t
1,973746711964372993t
2,1043031076787040257t
3,1066338147527741440t
4,963522018544152576t


In [57]:
label_columns = ['side-effect', 'ineffective', 'rushed', 'pharma', 'mandatory', 'unnecessary', 'political', 'ingredients', 'conspiracy', 'country', 'religious']

for i, prediction_row in enumerate(y_pred_filt):
    for j, label_column in enumerate(label_columns):
        test_raw_df.loc[i, label_column] = int(prediction_row[j])

test_raw_df[label_columns] = test_raw_df[label_columns].astype(int)
test_raw_df['none'] = (test_raw_df[label_columns] == 0).all(axis=1).astype(int)

In [58]:
test_raw_df.head(10)

,id,side-effect,ineffective,rushed,pharma,mandatory,unnecessary,political,ingredients,conspiracy,country,religious,none
0,1070378532260470789t,1,0,0,0,0,0,0,0,0,0,0,0
1,973746711964372993t,1,0,0,0,0,0,0,0,0,0,0,0
2,1043031076787040257t,1,0,0,0,0,0,0,0,0,0,0,0
3,1066338147527741440t,1,0,0,0,0,0,0,0,0,0,0,0
4,963522018544152576t,1,0,0,0,0,0,0,0,0,0,0,0
5,1027180081997930501t,1,0,0,0,0,0,0,0,0,0,0,0
6,1015587373537562625t,0,0,0,1,0,0,0,0,0,0,0,0
7,1010148037828726784t,1,0,0,0,0,0,0,0,0,0,0,0
8,1024938906750537728t,1,0,0,0,0,0,0,0,0,0,0,0
9,1087794890942099457t,1,0,0,0,0,0,0,0,0,0,0,0


In [59]:
test_raw_df['pred_classes'] = test_raw_df[label_columns].apply(lambda row: ' '.join([label_columns[i] for i, val in enumerate(row) if val == 1]), axis=1)
test_raw_df['pred_classes'] = test_raw_df['pred_classes'].apply(lambda labels: 'none' if len(labels) == 0 else labels)
test_raw_df = test_raw_df.drop(label_columns + ['none'], axis=1)

In [60]:
test_raw_df.head(20)

,id,pred_classes
0,1070378532260470789t,side-effect
1,973746711964372993t,side-effect
2,1043031076787040257t,side-effect
3,1066338147527741440t,side-effect
4,963522018544152576t,side-effect
5,1027180081997930501t,side-effect
6,1015587373537562625t,pharma
7,1010148037828726784t,side-effect
8,1024938906750537728t,side-effect
9,1087794890942099457t,side-effect


In [61]:
test_raw_df.to_csv('/content/drive/MyDrive/Colab Notebooks/FIRE_2023_OLD/DATA/Submissions/covid_bert.csv', index=False)

### F1 evaluation

In [ ]:
def count_metrics(y_pred, y_true):
    true_positives = 0
    true_negatives = 0
    false_positives = 0
    false_negatives = 0

    for true_labels, pred_labels in zip(y_true, y_pred):
        for true_label, pred_label in zip(true_labels, pred_labels):
            if true_label == 1 and pred_label == 1:
                true_positives += 1
            elif true_label == 0 and pred_label == 0:
                true_negatives += 1
            elif true_label == 0 and pred_label == 1:
                false_positives += 1
            elif true_label == 1 and pred_label == 0:
                false_negatives += 1

    return true_positives, true_negatives, false_positives, false_negatives


def evaluatef1(y_pred, y_true):
  tp, tn, fp, fn = count_metrics(y_true, y_pred)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1_micro = f1_score(y_true, y_pred, average='micro')
  f1_macro = f1_score(y_true, y_pred, average='macro')
  return precision, recall, f1_micro, f1_macro, tp, tn, fp, fn

In [ ]:
evaluatef1(y_pred_filt, y_true)

(0.6223404255319149,
 0.6842105263157895,
 0.6518105849582173,
 0.5992464101555011,
 117,
 1067,
 71,
 54)

In [ ]:
# @title
input_vectors = [[0.2, 0.5, 0.6], [0.4, 0.5, 0.7], [0.1, 0.6, 0.8]]
desired_outputs = [[0, 0, 0], [1, 0, 1], [0, 0, 1]]

threshold = find_best_thresholds(input_vectors, desired_outputs)
threshold

[0.202, 0.606, 0.606]

In [ ]:
# @title
input_vecs = [[0.2, 0.5, 0.6], [0.4, 0.5, 0.7], [0.1, 0.6, 0.8]]
thres = 0.5
filter_threshold(input_vecs, thres)

[[0, 1, 1], [0, 1, 1], [0, 1, 1]]

Evaluate Results

In [ ]:
def evaluate_classification(predicted_labels, true_labels):
  num_labels = len(true_labels[0])
  true_positives = [0] * num_labels
  true_negatives = [0] * num_labels
  false_positives = [0] * num_labels
  false_negatives = [0] * num_labels
  zero_labels_correct = 0
  total_zero_labels = 0

  for i in range(len(true_labels)):
      true_label = true_labels[i]
      predicted_label = predicted_labels[i]

      if true_label.nonzero()[0].size == 0:
        total_zero_labels += 1
      if true_label.nonzero()[0].size == 0 and predicted_label.nonzero()[0].size == 0:
          zero_labels_correct += 1

      for j in range(num_labels):
          if true_label[j] == 1 and predicted_label[j] == 1:
              true_positives[j] += 1
          elif true_label[j] == 0 and predicted_label[j] == 0:
              true_negatives[j] += 1
          elif true_label[j] == 0 and predicted_label[j] == 1:
                false_positives[j] += 1
          elif true_label[j] == 1 and predicted_label[j] == 0:
              false_negatives[j] += 1
  zero_labels_incorrect = total_zero_labels - zero_labels_correct
  return true_positives, true_negatives, false_positives, false_negatives, zero_labels_correct, zero_labels_incorrect

tp, tn, fp, fn, zero_labels_correct, zero_labels_incorrect = evaluate_classification(y_pred_filt, y_true)

print("True Positives:", tp)
print("True Negatives:", tn)
print("False Positives:", fp)
print("False Negatives:", fn)
print("Zero Labels Correct:", zero_labels_correct)
print("Zero Labels Incorrect:", zero_labels_incorrect)

True Positives: [28, 10, 4, 14, 11, 2, 6, 16, 16, 7, 3]
True Negatives: [78, 99, 105, 89, 105, 109, 102, 96, 80, 96, 108]
False Positives: [6, 7, 7, 13, 1, 2, 7, 3, 3, 5, 0]
False Negatives: [7, 3, 3, 3, 2, 6, 4, 4, 20, 11, 8]
Zero Labels Correct: 3
Zero Labels Incorrect: 5
